In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')

In [ ]:
df

In [ ]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [ ]:
df

In [ ]:
df.rename(columns={"v1":"target","v2":"message"},inplace=True)

In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['target']=encoder.fit_transform(df['target'])


In [ ]:
df

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df['message'][2]

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download("punkt")
nltk.download("stopwords")


chat_words_map = {
    'lol': 'laughing out loud', 'lmao': 'laughing my ass off', 'rofl': 'rolling on the floor laughing',
    'roflmao': 'rolling on the floor laughing my ass off', 'brb': 'be right back', 'ttyl': 'talk to you later',
    'idk': 'i do not know', 'omg': 'oh my god', 'btw': 'by the way', 'smh': 'shaking my head',
    'imo': 'in my opinion', 'imho': 'in my humble opinion', 'fyi': 'for your information',
    'afk': 'away from keyboard', 'asap': 'as soon as possible', 'bff': 'best friends forever',
    'g2g': 'got to go', 'gtg': 'got to go', 'np': 'no problem', 'nvm': 'never mind', 'wth': 'what the hell',
    'jk': 'just kidding', 'lmk': 'let me know', 'ikr': 'i know right', 'yolo': 'you only live once',
    'omfg': 'oh my freaking god', 'ty': 'thank you', 'thx': 'thanks', 'tysm': 'thank you so much',
    'plz': 'please', 'pls': 'please', 'u': 'you', 'ur': 'your', 'cya': 'see you',
    'xoxo': 'hugs and kisses', 'wyd': 'what are you doing', 'hbu': 'how about you',
    'bday': 'birthday', 'hbd': 'happy birthday', 'sup': 'what is up', 'cuz': 'because', 'coz': 'because',
    'dm': 'direct message', 'pm': 'private message', 'ftw': 'for the win', 'gr8': 'great',
    'luv': 'love', 'msg': 'message', 'txt': 'text', 'k': 'okay', 'okie': 'okay',
    'ya': 'yes', 'nah': 'no', 'yo': 'hey', 'bae': 'baby', 'fam': 'family'
}

stop_words = set(stopwords.words("english"))


def replace_chat_words(text):
    return ' '.join([chat_words_map.get(word.lower(), word) for word in text.split()])


def remove_tags(text):
    return re.sub(r'<.*?>', '', text)


def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))


def remove_stopwords_and_digits(text):
    text = re.sub(r'\d+', '', text)
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])


def preprocess_text(text):
    text = text.lower()
    text = replace_chat_words(text)
    text = remove_tags(text)
    text = remove_punctuation(text)
    text = remove_stopwords_and_digits(text)
    return text


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf=TfidfVectorizer()
X= tf.fit_transform(df['message'])
y=df['target']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score

model1=MultinomialNB()
model2=RandomForestClassifier()
model1.fit(X_train,y_train)
model2.fit(X_train,y_train)

y_pred1=model1.predict(X_test)
y_pred2=model2.predict(X_test)


print("Accuracy of model 1:",accuracy_score(y_test,y_pred1))
print("Accuracy of model 2:",accuracy_score(y_test,y_pred2))

In [ ]:
sample = ["Claim your free $500 Amazon gift card now! Only a few left! Text WIN to 90009!"]
sample_cleaned = [preprocess_text(msg) for msg in sample]
sample_vect = tf.transform(sample_cleaned)
result=model2.predict(sample_vect)

if (result == 1):
    print("Spam")
else:
    print("Not Spam")

In [ ]:
import joblib
joblib.dump(model2,'spam_model.pkl')
joblib.dump(tf,'vectorizer.pkl')